In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from PIL import Image
from sklearn.metrics import f1_score
import tensorflow as tf
import datetime

%load_ext autoreload
%autoreload 2
    
import ml_utils.road_seg as rs
import ml_utils.model as model
import ml_utils.data_augmentation as d_aug
import ml_utils.postprocess as pproc

In [ ]:
path_data = 'data'
path_train_dir = os.path.join(path_data, 'training')
path_test = os.path.join(path_data, 'test_set_images')
path_models = 'model'

# 2. Model training


In [ ]:
import ml_utils.model as model

m = model.Model(model_type='cnn_bn')

In [ ]:
m.train_model(path_train_dir, n_epoch=150, display_epoch=5, nmax=10, n_aug=400, n_worst=50, ratio=1.0)

In [ ]:
m.plot_stats('model/submission_model_stats.npy')

# 3. Results vizualisation


In [ ]:
file_save = 'model/submission_model.ckpt'

m = model.Model(model_type='cnn_bn')

In [ ]:
train_imgs, train_gt, val_imgs, val_gt = m._get_base_sets(ratio=0.8)
train_imgs, _, _ = rs.normalize_data(train_imgs, mode='all', mean_ref = m.mean, std_ref = m.std) 

In [ ]:
y_pred_train = m.apply_model(train_imgs, file_save)
print('F1-score train: {}'.format(m.predict_f1(train_gt, y_pred_train)))
#y_pred_valid = m.apply_model(val_imgs, file_save)
#print('F1-score validation: {}'.format(m.predict_f1(val_gt, y_pred_valid)))

In [ ]:
train_imgs, train_gt, val_imgs, val_gt = m._get_base_sets(ratio=1.0)
rs.display_predictions(y_pred_train, train_imgs, train_gt, n_display=20)

In [ ]:
pred_proc = pproc.process_all(y_pred_train)

print('F1-score train: {}'.format(m.predict_f1(train_gt, y_pred_train)))
print('F1-score train: {}'.format(m.predict_f1(train_gt, pred_proc)))

In [ ]:
#pred_proc = pproc.process_all(y_pred_valid)

#print('F1-score train: {}'.format(m.predict_f1(val_gt, y_pred_valid)))
#print('F1-score train: {}'.format(m.predict_f1(val_gt, pred_proc)))

# 4. Post Process

In [ ]:
test_img = rs.load_test_set(path_test)
test_img_norm, _, _ = rs.normalize_data(test_img, mode='all', mean_ref=m.mean, std_ref=m.std)

In [ ]:
# m = model.Model(model_type='cnn_bn', display_log=False)
y_pred_test = m.apply_model(test_img_norm, file_save, nmax=5)

In [ ]:
y_pred_test_proc = pproc.process_all(y_pred_test, size_image=608)

In [ ]:
im_proc = np.reshape(y_pred_test_proc, (-1, 608, 608))
im_base = np.reshape(y_pred_test, (-1, 608, 608))
rs.display_predictions(y_pred_test, test_img, im_proc, n_display=50)

# 4. Submission

In [ ]:
rs.create_submission(im_base, 'test_150epoch_pre.csv')

In [ ]:
rs.create_submission(im_proc, 'test_150epoch_full.csv')